In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [160]:
class salary_tax_calculator:
    """Tax calculation on salary as per Irish laws"""
    def __init__(self, pay):
        self.pay = pay
        self.tax_credits = 1650 + 1650 + 245 # pesonal tax credits exemption
        self.netPay = self.health_insurance = self.employee_pension_contributions = 0
        
    def set_params(self, employee_pension_contributions = 0, health_insurance = 0):        
        if health_insurance > 0:
            self.tax_credits += 200
        
        self.health_insurance = health_insurance * 12
        self.employee_pension_contributions = employee_pension_contributions/100 * self.pay
        self.pay += self.health_insurance
        self.PRSI = self.pay * 4/100 # Pay Related Social Insurance (PRSI) @ 4%
        self.netPay = self.pay - self.employee_pension_contributions
            
    def get_taxBracket(self):
        return [35300 * 0.8, (self.netPay - 35300) * 0.6] # 20% tax on 35,300 euros & 40% tax on remaining income
    
    def get_employee_pension_contributions(self):
        return self.employee_pension_contributions
        
    def get_USC_rates(self):
        url = 'https://www.citizensinformation.ie/en/money_and_tax/tax/income_tax/universal_social_charge.html'
        df = pd.read_html(url)[0]
        USC_rate = df['Rate'].apply(lambda x: float(x.strip('%'))/100)
        #USC_lim = df['Income band'].str.extract('([\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+)')
        return USC_rate
    
    def get_USC(self):
        USC_rate = self.get_USC_rates() # USC_rate = [0.5, 2, 4.5, 8]
        USC_lim = [12012, 20687, 70044] # USC_lim = int (USC_lim)

        if self.pay > USC_lim[2]:
            USC_tax = (USC_lim[2]-USC_lim[1])*USC_rate[2] + (self.pay-USC_lim[2])*USC_rate[3]
        else: 
            USC_tax = (self.pay-USC_lim[1])*USC_rate[2]
        
        return USC_lim[0]*USC_rate[0] + (USC_lim[1]-USC_lim[0])*USC_rate[1] + USC_tax # Universal Social Charge
        
    
    def income_after_tax(self):
        USC = self.get_USC()
        return sum(self.get_taxBracket()) + self.tax_credits - self.PRSI - USC

In [163]:
pay = 69000
p1 = salary_tax_calculator(pay)
care_select = 136.60
p1.set_params(10, care_select)
x = p1.income_after_tax()
#print(p1.get_employee_pension_contributions())
print('Income = ', x, x/12)

Income =  43720.710999999996 3643.392583333333
